# Kernel-based methods

> LS-SVM, Nystroem, Random Fourier Features, Gaussian Processes, ...

In [1]:
#| default_exp kernels

In [11]:
#| export
# from fastai.data.all import *
# from fastai.vision.all import *
from pathlib import Path
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

In [8]:
from uhina.loading import LoaderFactory

In [9]:
src = Path.home() / 'pro/data/woodwell-ringtrial/drive-download-20231013T123706Z-001'
loader = LoaderFactory.get_loader(src, 'ringtrial')
data = loader.load_data(analytes='potassium_cmolkg')
print(f'X shape: {data.X.shape}')

X shape: (1400, 1676)


In [10]:
X_subset = data.X[:10, :]

In [14]:
rbf_kernel(X_subset, X_subset).shape

(10, 10)

In [39]:
# high gamma -> more peaked
gamma = 1/data.X.shape[1]
print(f'default gamma: {gamma}')
gamma = 1/1000; gamma

default gamma: 0.0005966587112171838


0.001

In [40]:
# as gamma increase further points are considered more similar
np.exp(-gamma * np.sum((X_subset[0,:] - data.X[2, :])**2))

0.7570720129394493

In [20]:
rbf_kernel(X_subset, X_subset)[0,:]

array([1.        , 0.8314899 , 0.84700535, 0.97625302, 0.97344528,
       0.97546673, 0.97930349, 0.8925589 , 0.67826745, 0.97892596])

In [41]:
def nystrom_approximation(X, n_components, gamma=None):
    """
    Compute Nyström approximation for the given data.
    
    Args:
    X (np.array): Input data of shape (n_samples, n_features)
    n_components (int): Number of components to use for approximation
    gamma (float): Parameter for RBF kernel. If None, 1/n_features will be used.
    
    Returns:
    np.array: Nyström approximation of X
    """
    n_samples, n_features = X.shape
    
    # Randomly select subset of samples
    idx = np.random.choice(n_samples, n_components, replace=False)
    X_subset = X[idx]
    
    # Compute kernel between subset and all samples
    K_nm = rbf_kernel(X, X_subset, gamma=gamma)
    
    # Compute kernel for subset
    K_mm = K_nm[idx]
    
    # Compute eigendecomposition of K_mm
    eigvals, eigvecs = np.linalg.eigh(K_mm)
    
    # Avoid division by zero
    eigvals = np.maximum(eigvals, 1e-12)
    
    # Compute approximation
    tmp = np.dot(eigvecs / np.sqrt(eigvals), eigvecs.T)
    approx = np.sqrt(n_samples / n_components) * np.dot(K_nm, tmp)
    
    return approx

In [42]:
# Toy example using 1D soil spectra
# np.random.seed(42)
# Generate synthetic 1D soil spectra
n_samples = 1000

# n_wavelengths = 200
# X = np.random.rand(n_samples, n_wavelengths)

X = data.X

# Apply some transformations to make it more "spectrum-like"
# X = np.cumsum(X, axis=1)  # Make it monotonically increasing
# X += np.random.normal(0, 0.1, X.shape)  # Add some noise

# Compute Nyström approximation
n_components = 50
X_nystrom = nystrom_approximation(X, n_components)

print(f"Original shape: {X.shape}")
print(f"Nyström approximation shape: {X_nystrom.shape}")

Original shape: (1400, 1676)
Nyström approximation shape: (1400, 50)


In [ ]:

# Plot original spectra and their Nyström approximation
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title("Original Spectra")
plt.plot(X[:5].T)
plt.xlabel("Wavelength")
plt.ylabel("Intensity")

plt.subplot(1, 2, 2)
plt.title("Nyström Approximation")
plt.plot(X_nystrom[:5].T)
plt.xlabel("Component")
plt.ylabel("Value")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def LS_SVM(X, y, gamma=0.1, C=1.0):
    """
    Least Squares Support Vector Machine for regression
    
    Args:
    X (np.array): Input features of shape (n_samples, n_features)
    y (np.array): Target values of shape (n_samples,)
    gamma (float): RBF kernel parameter
    C (float): Regularization parameter
    
    Returns:
    tuple: (alpha, b) where alpha are the Lagrange multipliers and b is the bias term
    """
    n_samples = X.shape[0]
    
    # Compute the kernel matrix
    K = rbf_kernel(X, X, gamma=gamma)
    
    # Compute the solution
    H = K + np.eye(n_samples) / C
    H_inv = np.linalg.inv(H)
    alpha = H_inv.dot(y)
    b = np.mean(y - K.dot(alpha))
    
    return alpha, b

def predict_LS_SVM(X_train, X_test, alpha, b, gamma=0.1):
    """
    Make predictions using the trained LS-SVM model
    """
    K = rbf_kernel(X_test, X_train, gamma=gamma)
    return K.dot(alpha) + b

# Generate toy 1D spectral data
np.random.seed(42)
n_samples = 100
n_wavelengths = 50

# Generate wavelengths
wavelengths = np.linspace(400, 2400, n_wavelengths)

# Generate synthetic spectra
X = np.random.rand(n_samples, n_wavelengths)
X = np.cumsum(X, axis=1)  # Make it monotonically increasing
X += np.random.normal(0, 0.1, X.shape)  # Add some noise

# Generate target values (e.g., concentration of a chemical)
y = 3 * np.sin(X[:, 10]) + 2 * np.exp(-X[:, 30]) + np.random.normal(0, 0.1, n_samples)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train LS-SVM
alpha, b = LS_SVM(X_train, y_train, gamma=0.1, C=1.0)

# Make predictions
y_pred = predict_LS_SVM(X_train, X_test, alpha, b, gamma=0.1)

# Calculate RMSE
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
print(f"Root Mean Squared Error: {rmse:.4f}")

# Plot results
plt.figure(figsize=(12, 4))

# Plot example spectra
plt.subplot(121)
plt.title("Example Spectra")
plt.plot(wavelengths, X_train[:5].T)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Intensity")

# Plot true vs predicted values
plt.subplot(122)
plt.title("True vs Predicted Values")
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel("True Values")
plt.ylabel("Predicted Values")

plt.tight_layout()
plt.show()  